In [1]:
import pandas as pd
import numpy as np

In [4]:
bd = pd.read_csv(r'E:\papx\end_to_end_ml\nb_pr\scraping-bvl\Scraper\src\data\bvl_data.csv')

In [6]:
bd

,companyCode,companyName,shortName,nemonico,sectorCode,sectorDescription,lastDate,previousDate,buy,sell,...,negotiatedNationalAmount,operationsNumber,exderecho,percentageChange,currency,unity,segment,createdDate,numNeg,timestamp
0,73250,CREDICORP LTD.,Credicorp Ltd.,BAP,CD,DIVERSAS,2025-03-17T19:23:03,2025-03-14,185.0,192.00,...,1292974.0,16,191.99,1.93,US$,1,RV3,2025-03-17T15:20:17.622,0,2025-03-17T15:38:13.965658
1,73250,CREDICORP LTD.,Credicorp Ltd.,BAP,CD,DIVERSAS,2025-03-25T15:35:31,2025-03-24,194.9,196.19,...,3536.0,2,NaN,NaN,US$,1,RV3,2025-03-25T11:00:17.294,0,2025-03-25T11:02:58.644229
2,73250,CREDICORP LTD.,Credicorp Ltd.,BAP,CD,DIVERSAS,2025-03-25T17:21:24,2025-03-24,194.4,195.00,...,11332.0,3,194.63,1.34,US$,1,RV3,2025-03-25T13:00:17.833,0,2025-03-25T13:03:00.186771


In [7]:
bd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   companyCode               3 non-null      int64  
 1   companyName               3 non-null      object 
 2   shortName                 3 non-null      object 
 3   nemonico                  3 non-null      object 
 4   sectorCode                3 non-null      object 
 5   sectorDescription         3 non-null      object 
 6   lastDate                  3 non-null      object 
 7   previousDate              3 non-null      object 
 8   buy                       3 non-null      float64
 9   sell                      3 non-null      float64
 10  last                      2 non-null      float64
 11  minimun                   2 non-null      float64
 12  maximun                   2 non-null      float64
 13  opening                   2 non-null      float64
 14  previous      